In [1]:
import gspread
import pandas as pd
import arrow
import numpy as np

In [2]:
secrets_file = 'credentials.json'

### Import Sheet Data To Pandas

In [3]:
gc = gspread.service_account(filename=secrets_file)

In [4]:
g_sheet = gc.open_by_key('1chpKg4g_ReVQ4ciV2PxL_Y3S5hf5iy93OX3V6n7KdjQ')

In [5]:
worksheet = g_sheet.worksheet('Form responses 1')

In [6]:
df = pd.DataFrame(worksheet.get_all_records())

In [7]:
df.head(5)

,Timestamp,Date,How many pages of non fiction did you read today?,How many sections of Data Science did you cover today?,How many minutes did you meditate today?,How many programming sections did you cover today?,How many writing sections did you cover today?,How many minutes did you have focus today?,How many pages of fiction did you read today?,How many projects did you complete today?,How many programming problems did you solve today?,How many words did you write today
0,23/08/2022 14:44:00,15/08/2022,14,0,0,0,0,0,0,0,0,0
1,23/08/2022 14:44:56,16/08/2022,20,0,11,0,0,420,0,0,0,0
2,23/08/2022 14:47:10,17/08/2022,0,0,0,0,0,330,0,0,0,0
3,23/08/2022 14:47:54,18/08/2022,0,0,0,0,0,0,0,0,0,0
4,23/08/2022 14:48:39,19/08/2022,18,0,18,0,0,0,0,0,0,0


In [8]:
df = df.rename(columns={ 
    df.columns[2]: 'pages_nonfiction',
    df.columns[3]: 'sections_datascience',
    df.columns[4]: 'minutes_meditation',
    df.columns[5]: 'sections_programming',
    df.columns[6]: 'sections_writing',
    df.columns[7]: 'minutes_focus',
    df.columns[8]: 'pages_fiction',
    df.columns[9]: 'n_projects',
    df.columns[10]: 'n_problems',
    df.columns[11]: 'n_words',
})

#### Add A Datetime Column To Filter

In [9]:
def get_datetime_from_date(row):
    datetime_obj = arrow.get(row.Date, 'DD/MM/YYYY').datetime
    row['datetime'] = datetime_obj
    return row

In [10]:
df = df.apply(get_datetime_from_date, axis=1)

In [11]:
df.head(5)

,Timestamp,Date,pages_nonfiction,sections_datascience,minutes_meditation,sections_programming,sections_writing,minutes_focus,pages_fiction,n_projects,n_problems,n_words,datetime
0,23/08/2022 14:44:00,15/08/2022,14,0,0,0,0,0,0,0,0,0,2022-08-15 00:00:00+00:00
1,23/08/2022 14:44:56,16/08/2022,20,0,11,0,0,420,0,0,0,0,2022-08-16 00:00:00+00:00
2,23/08/2022 14:47:10,17/08/2022,0,0,0,0,0,330,0,0,0,0,2022-08-17 00:00:00+00:00
3,23/08/2022 14:47:54,18/08/2022,0,0,0,0,0,0,0,0,0,0,2022-08-18 00:00:00+00:00
4,23/08/2022 14:48:39,19/08/2022,18,0,18,0,0,0,0,0,0,0,2022-08-19 00:00:00+00:00


#### Add A Total Pages Column

In [12]:
def get_total_pages(row):
    total_pages = row['pages_fiction'] + row['pages_nonfiction']
    row['n_pages'] = total_pages
    return row

In [13]:
df = df.apply(get_total_pages, axis=1)

In [14]:
df.head(5)

,Timestamp,Date,pages_nonfiction,sections_datascience,minutes_meditation,sections_programming,sections_writing,minutes_focus,pages_fiction,n_projects,n_problems,n_words,datetime,n_pages
0,23/08/2022 14:44:00,15/08/2022,14,0,0,0,0,0,0,0,0,0,2022-08-15 00:00:00+00:00,14
1,23/08/2022 14:44:56,16/08/2022,20,0,11,0,0,420,0,0,0,0,2022-08-16 00:00:00+00:00,20
2,23/08/2022 14:47:10,17/08/2022,0,0,0,0,0,330,0,0,0,0,2022-08-17 00:00:00+00:00,0
3,23/08/2022 14:47:54,18/08/2022,0,0,0,0,0,0,0,0,0,0,2022-08-18 00:00:00+00:00,0
4,23/08/2022 14:48:39,19/08/2022,18,0,18,0,0,0,0,0,0,0,2022-08-19 00:00:00+00:00,18


In [15]:
def get_display_stats(df, statscol, average=False):
    dict_to_return = dict()

    this_sunday_datetime = arrow.now().ceil('week').datetime
    this_monday_datetime = arrow.now().floor('week').datetime

    this_month_end_datetime = arrow.now().ceil('month').datetime
    this_month_begin_datetime = arrow.now().floor('month').datetime

    last_sunday_datetime = arrow.now().ceil('week').shift(weeks=-1).datetime
    last_monday_datetime = arrow.now().floor('week').shift(weeks=-1).datetime

    last_month_end_datetime = arrow.now().ceil('month').shift(months=-1).datetime
    last_month_begin_datetime = arrow.now().floor('month').shift(months=-1).datetime

    this_week_df = df[(df.datetime >= this_monday_datetime) & (df.datetime <= this_sunday_datetime)]
    last_week_df = df[(df.datetime >= last_monday_datetime) & (df.datetime <= last_sunday_datetime)]

    this_week_col = this_week_df[statscol]
    last_week_col = last_week_df[statscol]

    this_month_df = df[(df.datetime >= this_month_begin_datetime) & (df.datetime <= this_month_end_datetime)]
    last_month_df = df[(df.datetime >= last_month_begin_datetime) & (df.datetime <= last_month_end_datetime)]

    this_month_col = this_month_df[statscol]
    last_month_col = last_month_df[statscol]

    this_week_dict = dict()
    this_week_dict['total'] = this_week_col.sum()
    
    last_week_dict = dict()
    last_week_dict['total'] = last_week_col.sum()

    this_month_dict = dict()
    this_month_dict['total'] = this_month_col.sum()

    last_month_dict = dict()
    last_month_dict['total'] = last_month_col.sum()

    total_dict = dict()
    total_dict['total'] = df[statscol].sum()

    if average == True:
        this_week_dict['average'] = np.nan_to_num(this_week_col.mean())
        last_week_dict['average'] = np.nan_to_num(last_week_col.mean())
        this_month_dict['average'] = np.nan_to_num(this_month_col.mean())
        last_month_dict['average'] = np.nan_to_num(last_month_col.mean())
        total_dict['average'] = np.nan_to_num(df[statscol].mean())

    dict_to_return['this_week'] = this_week_dict
    dict_to_return['last_week'] = last_week_dict
    dict_to_return['this_month'] = this_month_dict
    dict_to_return['last_month'] = last_month_dict
    dict_to_return['totals'] = total_dict    

    return dict_to_return

In [16]:
get_display_stats(df, 'minutes_focus', True)

{'this_week': {'total': 300, 'average': 75.0},
 'last_week': {'total': 340, 'average': 48.57142857142857},
 'this_month': {'total': 880, 'average': 58.666666666666664},
 'last_month': {'total': 2520, 'average': 157.5},
 'totals': {'total': 3460, 'average': 108.125}}

In [17]:
def get_chart_data(df):
    day_above = arrow.now().ceil('day').shift(days=-30).datetime
    
    last_30_days_df = df[(df.datetime >= day_above)]

    return last_30_days_df

In [18]:
last_30_days_df = get_chart_data(df)

In [19]:
last_30_days_df

,Timestamp,Date,pages_nonfiction,sections_datascience,minutes_meditation,sections_programming,sections_writing,minutes_focus,pages_fiction,n_projects,n_problems,n_words,datetime,n_pages
2,23/08/2022 14:47:10,17/08/2022,0,0,0,0,0,330,0,0,0,0,2022-08-17 00:00:00+00:00,0
3,23/08/2022 14:47:54,18/08/2022,0,0,0,0,0,0,0,0,0,0,2022-08-18 00:00:00+00:00,0
4,23/08/2022 14:48:39,19/08/2022,18,0,18,0,0,0,0,0,0,0,2022-08-19 00:00:00+00:00,18
5,23/08/2022 14:49:07,20/08/2022,0,0,0,0,0,0,0,0,0,0,2022-08-20 00:00:00+00:00,0
6,23/08/2022 14:49:25,21/08/2022,0,0,0,0,0,0,0,0,0,0,2022-08-21 00:00:00+00:00,0
7,23/08/2022 14:49:48,22/08/2022,0,0,0,0,0,0,0,0,0,0,2022-08-22 00:00:00+00:00,0
8,24/08/2022 00:17:33,23/08/2022,28,0,0,0,0,320,0,0,0,0,2022-08-23 00:00:00+00:00,28
9,25/08/2022 00:36:33,24/08/2022,30,0,11,0,0,330,0,0,0,0,2022-08-24 00:00:00+00:00,30
10,26/08/2022 00:33:24,25/08/2022,23,0,12,0,0,210,18,0,0,0,2022-08-25 00:00:00+00:00,41
11,27/08/2022 12:29:31,26/08/2022,35,0,12,0,0,200,0,0,0,0,2022-08-26 00:00:00+00:00,35
